<a href="https://colab.research.google.com/github/LenkaMikova/NoisMaps/blob/main/DP_LMI_dataDown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIPLOMOVÁ PRÁCE
## Hluková mapa odvozená ze satelitních dat

# 1. Úvodní




## 1.1 Instalace a importy

### 1.1.1 Instalace balíčků

Postačí jednou!

In [ ]:
!pip install -U geemap

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


Postačí jednou!

In [ ]:
!pip install earthengine-api
# Pri problemech s instalaci se doporucuje odinstalovat a znovu nainstalovat
#!pip uninstall earthengine-api -y

Musí se instalovat pokaždé!

In [1]:
!pip install openeo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.0/271.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.1 MB/s eta 0:00:00


In [ ]:
#!pip install sentinelhub

Postačí jednou!

In [ ]:
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00


### 1.1.2 Import balíčků

In [2]:
import os

In [3]:
import ee

In [4]:
import geemap

In [5]:
import openeo

In [6]:
import geopandas as gpd

In [7]:
import pandas as pd

In [8]:
import time

In [9]:
import geemap.core as geemapCore

In [10]:
from shapely.geometry import shape, mapping, Polygon, MultiPolygon, box

In [11]:
import datetime

In [ ]:
#import fiona

In [ ]:
#import sentinelhub

### 1.1.3 Nastavení výstupního adresáře

In [12]:
# Polygon oblasti sberu dat (dostupna data ze SHM 2022) v SHP (cesta k adresari)
shp_dir = '/content/drive/MyDrive/DP'

# Vystupni soubory a adresare - cesta
out_path = '/content/drive/MyDrive/DP/output'
out_dir_L8 = '/content/drive/MyDrive/DP/output/L8_output'
out_dir_S5P = '/content/drive/MyDrive/DP/output/S5P_output'

if not os.path.exists(out_path):
    os.makedirs(out_path)

## 1.2 Připojení externích služeb

### 1.2.1 Google Drive

In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### 1.2.2 Google Earth Engine

In [14]:
ee.Authenticate()
print("Authenticate successful")

Authenticate successful


In [15]:
# V pripade problemu je nutno v GEE přímo v účtu registrovat projekt (vpravo nahore)
try:
    ee.Initialize(project='lenkamikova256')
    print("Initialization successful")
except ee.EEException as e:
    print("Initialization failed: ", e)

Initialization successful


### 1.2.3 Sentinel *Hub*

In [ ]:
#from sentinelhub import SHConfig

#config = SHConfig()

# Nastaveni prihlasovacich udaju
#config.instance_id = 'lenkamikova256'
#config.sh_client_id = '23b04f7a-dfc5-4e65-a324-0a25f882f4cc'
#config.sh_client_secret = 'AJiyRTpgwdRhtmQIZQQFQc6FisJTR3dc'

# Overeni spravnosti nastaveni
#if not config.instance_id or not config.sh_client_id or not config.sh_client_secret:
#    print("Warning: Missing credentials for Sentinel Hub services")

Vytvoření připojení a získání seznamu kolekcí - k dispozici pouze Sentinel 1 a Sentinel 2.

In [ ]:
 #from sentinelhub import SentinelHubCatalog

# Inicializace připojení
#catalog = SentinelHubCatalog(config=config)

# Získání seznamu kolekcí
#collections = catalog.get_collections()
#collection_ids = [collection['id'] for collection in collections]

#print("Available collection IDs:")
#for collection_id in collection_ids:
 #   print(collection_id)

# 2. Satelitní data

## 2.1 Vstupní data a parametry

### 2.1.1 Nastavení vstupních parametrů

In [16]:
startDate = '2022-03-01' # pocatecni datum sberu dat
endDate = '2022-10-31'  # konecne datu sberu dat
center = [49.926472, 14.861755]
zoom = 10

### 2.1.2 Nastavení vstupních dat

Vstupní data z GeoJSON

In [ ]:
# Polygon oblasti sberu dat (dostupna data ze SHM 2022) v GeoJSON (cesta k adresari)
#geojson_path = 'content/drive/MyDrive/DP/oblast_hlukData.geojson';
#gdf = gpd.read_file(geojson_path)

Vstupní data ze ShapeFile

In [17]:
# Test, zda soubor existuje a kontrola ostatních souborů
required_files = ['oblast_hlukData.shp', 'oblast_hlukData.shx', 'oblast_hlukData.dbf', 'oblast_hlukData.prj']
missing_files = [f for f in required_files if not os.path.exists(os.path.join(shp_dir, f))]

if missing_files:
    raise FileNotFoundError(f"Missing required shapefile components: {missing_files}")

shp_path = os.path.join(shp_dir, 'oblast_hlukData.shp')

try:
    gdf = gpd.read_file(shp_path)
    print("SHP file loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Error loading SHP file: {e}")

SHP file loaded successfully.


Kontrola geometrie

In [ ]:
if gdf.empty:
    raise ValueError("Geodataframe is empty. Check the input shapefile.")

polygon = gdf.geometry.iloc[0]
print("Geometry loaded successfully.")

Geometry loaded successfully.


Převod na GeoJSON pro GEE



In [ ]:
# Pred prevodem je treba provest autentifikaci a inicializace GEE (cela oblast)
try:
    oblast = geemap.gdf_to_ee(gdf)
    print("Geometry converted to Earth Engine format successfully.")
except Exception as e:
    raise RuntimeError(f"Error converting geometry to Earth Engine format: {e}")

Geometry converted to Earth Engine format successfully.


Rozdělení polygonu na menší dlaždice z důvodu vysoké výpočetní náročnosti při volbě celého území.

In [ ]:
def split_polygon(polygon, rows, cols):
    minx, miny, maxx, maxy = polygon.bounds
    width = (maxx - minx) / cols
    height = (maxy - miny) / rows
    tiles = []
    for i in range(cols):
        for j in range(rows):
            minx_tile = minx + i * width
            miny_tile = miny + j * height
            maxx_tile = minx + (i + 1) * width
            maxy_tile = miny + (j + 1) * height
            tile = box(minx_tile, miny_tile, maxx_tile, maxy_tile)
            tiles.append(tile)
    return tiles

Rozdělení oblasti na mřížku 4x4

In [ ]:
#tiles = split_polygon(polygon, 4,4) # zvolit vhodne z duvodu narocnosti pri exportu dat
#tiles = split_polygon(polygon, 8,8)
tiles = split_polygon(polygon, 16,16)
print("Tiles is set up to 16 x 16")


Tiles is set up to 16 x 16


Vytvoření GeoDataFrame pro dlaždice

In [ ]:
tiles_gdf = gpd.GeoDataFrame({'geometry': tiles}, crs=gdf.crs)
print("GeoData Frame created.")

GeoData Frame created.


Validace pro kontrolu a opravu platnosti geometrie

In [ ]:
def validate_geometry(gdf):
    valid_gdf = gdf[gdf.is_valid]
    return valid_gdf

In [ ]:
tiles_gdf = validate_geometry(tiles_gdf)
print("Geometry GeoData Frame is valid.")

Geometry GeoData Frame is valid.


Převod dlaždic pro GEE

In [ ]:
tiles_ee = [geemap.geopandas_to_ee(tiles_gdf) for tile_gdf in tiles_gdf.geometry]
print("Tiles Converted for GEE.")

Tiles Converted for GEE.


## 2.2 LANDSAT 8

Funkce na přejmenování bandů pro Landsat 8

In [ ]:
def rename_bands(image):
    return image.select(
        ['SR_B7', 'SR_B6', 'SR_B5', 'SR_B4', 'SR_B3', 'SR_B2'],
        ['SWIR2', 'SWIR1', 'NIR', 'red', 'green', 'blue']
    )

### 2.2.1 Indexy

#### **NDVI** přes GEE

In [ ]:
def calculate_ndvi(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    ndvi = landsat_collection \
        .map(lambda image: image.normalizedDifference(['NIR', 'red']).rename('NDVI')) \
        .median() \
        .clip(tile)

    return ndvi

In [ ]:
ndvi_tiles = [calculate_ndvi(tile, startDate, endDate) for tile in tiles_ee]
print("NDVI calculated.")


NDVI calculated.


Sloučení NDVI dlaždic do jednoho obrazu

In [ ]:
ndvi_mosaic = ee.ImageCollection(ndvi_tiles).mosaic()

NDVImap = geemap.Map(center=center, zoom=zoom)
NDVImap.addLayer(ndvi_mosaic, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI Mosaic')
NDVImap.setCenter(14.861755, 49.926472, zoom)
NDVImap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **NDBI** přes GEE

In [ ]:
def calculate_ndbi(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    ndbi = landsat_collection \
        .map(lambda image: image.normalizedDifference(['SWIR1', 'NIR']).rename('NDBI')) \
        .median() \
        .clip(tile)

    return ndbi

In [ ]:
ndbi_tiles = [calculate_ndbi(tile, startDate, endDate) for tile in tiles_ee]
print("NDBI calculated.")

NDBI calculated.


In [ ]:
ndbi_mosaic = ee.ImageCollection(ndbi_tiles).mosaic()

NDBImap = geemap.Map(center=center, zoom=zoom)
NDBImap.addLayer(ndbi_mosaic, {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'yellow']}, 'NDBI Mosaic')
NDBImap.setCenter(14.861755, 49.926472, zoom)
NDBImap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **MNDWI** přes GEE

In [ ]:
def calculate_mndwi(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    mndwi = landsat_collection \
        .map(lambda image: image.normalizedDifference(['green', 'SWIR1']).rename('MNDWI')) \
        .median() \
        .clip(tile)

    return mndwi

In [ ]:
mndwi_tiles = [calculate_mndwi(tile, startDate, endDate) for tile in tiles_ee]
print("MNDVW calculated.")

MNDVW calculated.


In [ ]:
mndwi_mosaic = ee.ImageCollection(mndwi_tiles).mosaic()

MNDWImap = geemap.Map(center=center, zoom=zoom)
MNDWImap.addLayer(mndwi_mosaic, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'MNDWI Mosaic')
MNDWImap.setCenter(14.861755, 49.926472, zoom)
MNDWImap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **UI** přes GEE

In [ ]:
def calculate_ui(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    ui = landsat_collection \
        .map(lambda image: image.expression(
            '((NIR - (Red + Green)) / (NIR + (Red + Green)))',
            {
                'NIR': image.select('NIR'),
                'Red': image.select('red'),
                'Green': image.select('green')
            }).rename('UI')
        ) \
        .median() \
        .clip(tile)

    return ui

In [ ]:
ui_tiles = [calculate_ui(tile, startDate, endDate) for tile in tiles_ee]
print("UI calculated.")

UI calculated.


In [ ]:
ui_mosaic = ee.ImageCollection(ui_tiles).mosaic()

UImap = geemap.Map(center=center, zoom=zoom)
UImap.addLayer(ui_mosaic, {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'orange']}, 'UI Mosaic')
UImap.setCenter(14.861755, 49.926472, zoom)
UImap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **ISA** přes GEE

In [ ]:
def calculate_isa(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    isa = landsat_collection \
        .map(lambda image: image.normalizedDifference(['SWIR1', 'NIR']).rename('ISA')) \
        .median() \
        .clip(tile)

    return isa

In [ ]:
isa_tiles = [calculate_isa(tile, startDate, endDate) for tile in tiles_ee]
print("ISA calculated.")

ISA calculated.


In [ ]:
isa_mosaic = ee.ImageCollection(isa_tiles).mosaic()

ISAmap = geemap.Map(center=center, zoom=zoom)
ISAmap.addLayer(isa_mosaic, {'min': -1, 'max': 1, 'palette': ['black', 'white', 'brown']}, 'ISA Mosaic')
ISAmap.setCenter(14.861755, 49.926472, zoom)
ISAmap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

### 2.2.2 Výstupní soubor indexů do CSV

Skript kombinuje několik snímků do jedné mozaiky, která představuje sjednocený obraz, kde jsou vybrány nejlepší hodnoty pixelů z překrývajících se snímků.

In [ ]:
ndvi_mosaic = ee.ImageCollection(ndvi_tiles).mosaic()
print("Mosaic NDVI created.")
ndbi_mosaic = ee.ImageCollection(ndbi_tiles).mosaic()
print("Mosaic NDBI created.")
mndwi_mosaic = ee.ImageCollection(mndwi_tiles).mosaic()
print("Mosaic MNDWI created.")
ui_mosaic = ee.ImageCollection(ui_tiles).mosaic()
print("Mosaic UI created.")
isa_mosaic = ee.ImageCollection(isa_tiles).mosaic()
print("Mosaic ISA created.")

Mosaic NDVI created.
Mosaic NDBI created.
Mosaic MNDWI created.
Mosaic UI created.
Mosaic ISA created.


In [ ]:
landsat_combined = ee.Image.cat([ndvi_mosaic.rename('NDVI'),
                         ndbi_mosaic.rename('NDBI'),
                         mndwi_mosaic.rename('MNDWI'),
                         ui_mosaic.rename('UI'),
                         isa_mosaic.rename('ISA')])
print("The combination of several images (frames) from different indexes into one multi-band image is complete.")

The combination of several images (frames) from different indexes into one multi-band image is complete.


## 2.3 SENTINEL 5P

*Jelikož kolekce ze Sentinel Hub neobsahuje Sentinel 5P, tak je třeba data stahovat prostřednictvím GEE.*

Načtení Image Collection



In [ ]:
tiles_S5P = tiles # pouzit tiles, ktery byl u Landsat 8
tiles_gdf = gpd.GeoDataFrame({'geometry': tiles_S5P}, crs=gdf.crs)

Data jsou průměry za vybrané období.

In [ ]:
try:
    sentinel5p = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2') \
                   .filterDate(startDate, endDate) \
                   .filterBounds(geemap.geopandas_to_ee(gdf))

    CO = sentinel5p.select("CO").mean().rename('CO')
    HCHO = sentinel5p.select("HCHO").mean().rename('HCHO')
    NO2 = sentinel5p.select("NO2").mean().rename('NO2')
    O3 = sentinel5p.select(["O3_PR", "O3_TCL"]).mean().rename('O3')
    SO2 = sentinel5p.select("SO2").mean().rename('SO2')
    CH4 = sentinel5p.select("CH4").mean().rename('CH4')

    sentinel_combined = ee.Image.cat([CO, HCHO, NO2, O3, SO2, CH4])

except Exception as e:
    print(f"Error loading or processing Sentinel-5P ImageCollection: {e}")

Error loading or processing Sentinel-5P ImageCollection: name 'ee' is not defined


In [ ]:
def check_tasks(tasks):

    # Běží, dokud některá úloha zůstává aktivní
    while any(task.active() for task in tasks):
        for i, task in enumerate(tasks):
            if task.active():
                print(f'Task {i} is still running...')
        time.sleep(30)  # Zkontroluje stav každých 30 sekund
    print('All tasks have completed.')

    # Po dokončení všech úloh se zobrazí jejich stav
    for i, task in enumerate(tasks):
        task_state = task.status().get("state")
        print(f'Task {i} has state: {task_state}')
        if task_state != 'COMPLETED':
            print(f"Task {i} failed or was canceled with details: {task.status().get('error_message')}")

In [ ]:
def visualize_band_S5P(band_name, palette, vmin, vmax, center, zoom):
    band_mosaic = sentinel_combined.select(band_name)
    band_map = geemap.Map(center=center, zoom=zoom)
    band_map.addLayer(band_mosaic, {'min': vmin, 'max': vmax, 'palette': palette}, f'{band_name} Mosaic')
    band_map.setCenter(center[1], center[0], zoom)
    return band_map

In [ ]:
palette = ['blue', 'white', 'red']  # Barevná paleta pro CO (modrá až červená)
vmin = 0  # Minimální hodnota pro vizualizaci
vmax = 0.05  # Maximální hodnota pro vizualizaci (hodnoty se mohou lišit podle dat)
scale = 1000

Funkce pro vizualizaci - nutno rozdělit na části z důvodu vykreslení velkého množství dat najednou.

In [ ]:
def visualize_tile_S5P(tile, band_name, palette, vmin, vmax, center, zoom, scale=2000):
    tile_ee = geemap.geopandas_to_ee(gpd.GeoDataFrame(geometry=[tile], crs=tiles_gdf.crs))
    band_mosaic = sentinel_combined.select(band_name).clip(tile_ee.geometry())

    band_map = geemap.Map(center=center, zoom=zoom)
    band_map.addLayer(band_mosaic, {'min': vmin, 'max': vmax, 'palette': palette, 'scale': scale}, f'{band_name} Mosaic')
    band_map.setCenter(center[1], center[0], zoom)
    return band_map


#### **CO** přes GEE

In [ ]:
for i, tile in enumerate(tiles_S5P[:2]): # pouze prvni 2 dlazdice
    print(f"Rendering tiles {i+1}/{len(tiles_S5P)}")
    tile_map = visualize_tile_S5P(tile, CO, palette, vmin, vmax, center, zoom)
    display(tile_map)
    time.sleep(1)  # Pauza mezi vykreslováním dlaždic

Rendering tiles 1/256


In [ ]:
CO_mosaic = CO

CO_map = geemap.Map(center=center, zoom=zoom)
CO_map.addLayer(CO_mosaic, {vmin, vmax, palette}, 'CO Mosaic')
CO_map.setCenter(14.861755, 49.926472, zoom)
CO_map

NameError: name 'CO' is not defined

#### **HCHO** přes GEE

In [ ]:
HCHO_mosaic = HCHO

HCHO_map = geemap.Map(center=center, zoom=zoom)
HCHO_map.addLayer(HCHO_mosaic, {vmin, vmax, palette}, 'HCHO Mosaic')
HCHO_map.setCenter(14.861755, 49.926472, zoom)
HCHO_map

#### **NO2** přes GEE

In [ ]:
NO2_mosaic = NO2

NO2_map = geemap.Map(center=[49.926472, 14.861755], zoom=10)
NO2_map.addLayer(NO2_mosaic, {'min': 0, 'max': 0.05, 'palette': ['blue', 'white', 'yellow']}, 'NO2 Mosaic')
NO2_map.setCenter(14.861755, 49.926472, 10)
NO2_map

#### **O3** přes GEE

In [ ]:
O3_mosaic = O3

O3_map = geemap.Map(center=[49.926472, 14.861755], zoom=10)
O3_map.addLayer(O3_mosaic, {'min': 0, 'max': 0.1, 'palette': ['blue', 'white', 'purple']}, 'O3 Mosaic')
O3_map.setCenter(14.861755, 49.926472, 10)
O3_map

#### **SO2** přes GEE

In [ ]:
SO2_mosaic = SO2

SO2_map = geemap.Map(center=[49.926472, 14.861755], zoom=10)
SO2_map.addLayer(SO2_mosaic, {'min': 0, 'max': 0.05, 'palette': ['blue', 'white', 'orange']}, 'SO2 Mosaic')
SO2_map.setCenter(14.861755, 49.926472, 10)
SO2_map

#### **CH4** přes GEE

In [ ]:
CH4_mosaic = CH4

CH4_map = geemap.Map(center=[49.926472, 14.861755], zoom=10)
CH4_map.addLayer(CH4_mosaic, {'min': 0, 'max': 0.05, 'palette': ['blue', 'white', 'brown']}, 'CH4 Mosaic')
CH4_map.setCenter(14.861755, 49.926472, 10)
CH4_map

## 2.4 SENTINEL 2

### Spuštění kolekce Sentinel 2 přes EO Browser

In [ ]:
#connection.describe_collection("SENTINEL2_L2A")

In [ ]:
#s2cube = connection.load_collection(
    #"SENTINEL2_L2A",
    #spatial_extent={"west":12.09, "south":51.06, "east":18.87, "north":48.55},
    #temporal_extent=["2022-06-01", "2022-10-01"], # spatial_extent je Česká republika
    #bands=["B02","B04", "B08","SLC"], # SLC je band s cloud cover
    #max_cloud_cover=80,

)

NDVI

*Greenest (mozaika ze vsech snímku z celeho roku a vybraní toho nejzelenějšího) *

In [ ]:
#blue = s2cube.band("B02")*10000
#red = s2cube.band("B04")*10000
#NIR = s2cube.band("B08")*10000
#slc = s2cube.band("SLC")

Odmaskuje vše co není class 4 (vegetace)

In [ ]:
#mask = (slc != 4)

EVI

In [ ]:
#evi = 2.5 * (NIR - red)/(NIR + 6* red - 7.5* blue +1)

*temporal maximum value for each pixel a download*

In [ ]:
#evi.max_time().download("evi_composite.tiff")

EVI včetně masky

In [ ]:
#mask_resampled = mask.resample_sube_spatial(evi)
#evi_cube_mask = evi.mask(mask_resampled)
#evi_cube_mask.max_time().download("evi_mask.tiff")

# 3. Výstupy

## 3.1 Export výsledků do CSV

#### Indexy z Landsat 8 v1


***VARIANTA 1*** - EXPORT CELÉHO TILE

Funkce pro export celého tile z L8, může však způsobit problém při exportu a bude třeba spíše zvolit variantu snížení počtu pixelů anebo funkce **export_tile_reduce_L8**



In [ ]:
def export_tile_L8(tile, index):
  try:
    tile_ee = geemap.geopandas_to_ee(gpd.GeoDataFrame(geometry=[tile], crs=tiles_gdf.crs))

    # Vytvoření souřadnicové mřížky a extrahování hodnot
    latlon = ee.Image.pixelLonLat().addBands(landsat_combined)
    sample = latlon.sample(
        region=tile_ee.geometry(),
        scale=30,
        numPixels=1e8, # možnost snízení poctu pixelů (napr. 5e5 z 1e8)
        geometries=True
    )
    task = ee.batch.Export.table.toDrive(
            collection=sample,
            description=f'Landsat export - part {index}',
            folder='L8_output',
            fileNamePrefix=f'landsat_indices_part_{index}',
            fileFormat='CSV'
        )
    task.start()
    return task

  except Exception as e:
        print(f"Error creating export task for tile {index}: {e}")
        return None

In [ ]:
tasks = [export_tile_L8(tile, i) for i, tile in enumerate(tiles) if tile is not None]

#### Indexy z Landsat 8 v2


***VARIANTA 2*** - EXPORT ČÁSTÍ TILE

In [ ]:
def export_tile_reduce_L8(tile, index):
    # Převedení dlaždice na Earth Engine formát
    try:
        tile_ee = geemap.geopandas_to_ee(gpd.GeoDataFrame(geometry=[tile], crs=tiles_gdf.crs))
    except Exception as e:
        print(f"Error converting tile {index} to EE format: {e}")
        return None

    # Výpočet průměrné hodnoty pro každou dlaždici pomocí reduceRegion
    try:
        reduced = landsat_combined.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=tile_ee.geometry(),
            scale=30,
            maxPixels=1e4,
            bestEffort=True  # Použití bestEffort, pokud je příliš mnoho pixelů
        )

        # Pokud reduceRegion nevrátí žádná data, přerušíme funkci
        if not reduced.getInfo():
            print(f"No data returned for tile {index}")
            return None

    except Exception as e:
        print(f"Error reducing region for tile {index}: {e}")
        return None

    # Vytvoření feature pro export
    try:
        feature = ee.Feature(None, reduced)
        collection = ee.FeatureCollection([feature])

        # Export do CSV
        task = ee.batch.Export.table.toDrive(
            collection=collection,
            description=f'Export of landsat indices with coordinates - part {index}',
            folder='L8_output',
            fileNamePrefix=f'landsat_indices_reduce_part_{index}',
            fileFormat='CSV'
        )
        task.start()
        return task

    except Exception as e:
        print(f"Error creating export task for tile {index}: {e}")
        return None

In [ ]:
def check_tasks(tasks):
    while any(task.active() for task in tasks):
        for i, task in enumerate(tasks):
            if task.active():
                print(f'Task {i} is still running...')
        time.sleep(30)
    print('All tasks have completed.')
    for i, task in enumerate(tasks):
        print(f'Task {i} has state: {task.status().get("state")}')

Kontrola stavu jednotlivých úloh

In [ ]:
tasks = [export_tile_reduce_L8(tile, i) for i, tile in enumerate(tiles) if tile is not None]
check_tasks(tasks)

All tasks have completed.
Task 0 has state: COMPLETED
Task 1 has state: COMPLETED
Task 2 has state: COMPLETED
Task 3 has state: COMPLETED
Task 4 has state: COMPLETED
Task 5 has state: COMPLETED
Task 6 has state: COMPLETED
Task 7 has state: COMPLETED
Task 8 has state: COMPLETED
Task 9 has state: COMPLETED
Task 10 has state: COMPLETED
Task 11 has state: COMPLETED
Task 12 has state: COMPLETED
Task 13 has state: COMPLETED
Task 14 has state: COMPLETED
Task 15 has state: COMPLETED
Task 16 has state: COMPLETED
Task 17 has state: COMPLETED
Task 18 has state: COMPLETED
Task 19 has state: COMPLETED
Task 20 has state: COMPLETED
Task 21 has state: COMPLETED
Task 22 has state: COMPLETED
Task 23 has state: COMPLETED
Task 24 has state: COMPLETED
Task 25 has state: COMPLETED
Task 26 has state: COMPLETED
Task 27 has state: COMPLETED
Task 28 has state: COMPLETED
Task 29 has state: COMPLETED
Task 30 has state: COMPLETED
Task 31 has state: COMPLETED
Task 32 has state: COMPLETED
Task 33 has state: COMPLETE

Seznam cest ke všem CSV souborům se spektrálními indexy (NDVI, NDBI, MNDWI, UI, ISA).

In [ ]:
csv_files = [os.path.join(out_dir_L8, f'landsat_indices_reduce_part_{i}.csv') for i in range(len(tiles))]

In [ ]:
missing_files = [file for file in csv_files if not os.path.exists(file)]
if missing_files:
    print("The following files are missing:")
    for missing_file in missing_files:
        print(missing_file)
else:
    print("All files exist, we continue to load and merge.")

    # Načtení a sloučení CSV souborů
    try:
        df_list = [pd.read_csv(file) for file in csv_files if os.path.exists(file)]
        df_combined = pd.concat(df_list)

        # Uložení kombinovaného DataFrame do jednoho CSV souboru
        combined_csv_path = os.path.join(out_path, 'landsat_indices_combined.csv')
        df_combined.to_csv(combined_csv_path, index=False)
        print(f"The combined CSV file was successfully saved to: {combined_csv_path}")

    except Exception as e:
        print(f"Error loading and merging files: {e}")

Následující soubory chybí:
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_0.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_1.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_2.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_3.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_4.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_5.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_6.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_7.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_8.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_9.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_10.csv
/content/drive/MyDrive/DP/output/L8_output/landsat_indices_reduce_part_11.csv
/content/drive/MyDrive/DP/output/L8_output/land

In [ ]:
try:
    # Načtení jednotlivých CSV souborů do seznamu DataFrame
    df_list = [pd.read_csv(file) for file in csv_files if os.path.exists(file)]

    # Sloučení všech DataFrame do jednoho
    df_combined = pd.concat(df_list, ignore_index=True)

    # Uložení sloučeného DataFrame do jednoho CSV souboru
    df_combined.to_csv(out_dir_L8, index=False)
    print(f"The combined CSV file was successfully saved to: {out_dir_L8}")

except Exception as e:
    print(f"Error loading and merging files: {e}")

Error loading and merging files: name 'csv_files' is not defined


#### Data ze Sentinel 5P

VARIANTA 1 - EXPORT CELÉHO TILE

In [ ]:
def export_tile_S5P(tile, index):
    tile_ee = geemap.geopandas_to_ee(gpd.GeoDataFrame(geometry=[tile], crs=tiles_gdf.crs))
    latlon = ee.Image.pixelLonLat().addBands(sentinel_combined)
    sample = latlon.sample(
        region=tile_ee.geometry(),
        scale=30,
        numPixels=1e6,
        geometries=True
    )
    task = ee.batch.Export.table.toDrive(
        collection=sample,
        description=f'Export of sentinel indices with coordinates - part {index}',
        folder='output',
        fileNamePrefix=f'sentinel_indices_part_{index}',
        fileFormat='CSV'
    )
    task.start()
    return task

In [ ]:
tasks = [export_tile_S5P(tile, i) for i, tile in enumerate(tiles)]
check_tasks(tasks)

EEException: Request payload size exceeds the limit: 10485760 bytes.

VARIANTA 2 - EXPORT ČÁSTÍ TILE

In [ ]:
def export_tile_reduce_S5P(tile, index):
    try:
        tile_ee = geemap.geopandas_to_ee(gpd.GeoDataFrame(geometry=[tile], crs=tiles_gdf.crs))
    except Exception as e:
        print(f"Error converting tile {index} to EE format: {e}")
        return None

    try:
        reduced = sentinel_combined.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=tile_ee.geometry(),
            scale=1000,
            maxPixels=1e5,
            bestEffort=True
        )

        if not reduced.getInfo():
            print(f"No data returned for tile {index}")
            return None

    except Exception as e:
        print(f"Error reducing region for tile {index}: {e}")
        return None

    try:
        feature = ee.Feature(None, reduced)
        collection = ee.FeatureCollection([feature])

        task = ee.batch.Export.table.toDrive(
            collection=collection,
            description=f'Export of Sentinel-5P indices with coordinates - part {index}',
            folder='output',
            fileNamePrefix=f'sentinel_indices_reduce_part_{index}',
            fileFormat='CSV'
        )
        task.start()
        return task

    except Exception as e:
        print(f"Error creating export task for tile {index}: {e}")
        return None

In [ ]:
tasks = [export_tile_reduce_S5P(tile, i) for i, tile in enumerate(tiles_S5P) if tile is not None]
check_tasks(tasks)

In [ ]:
csv_files = [os.path.join(out_dir_S5P, f'sentinel_indices_reduce_part_{i}.csv') for i in range(len(tiles))]

# Sloučení všech CSV souborů do jednoho DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
df_combined = pd.concat(df_list)

# Uložení kombinovaného DataFrame do jednoho CSV souboru
df_combined.to_csv(out_dir_S5P, index=False)